In [1]:
import pandas as pd
import pickle
import numpy as np
import tqdm

get_all_games = 0
start_val = 500

In [2]:
if get_all_games:
    query = """
    select distinct game_ref_id, date from (
    select distinct game_ref_id, updated_timestamp, dt as date
    from `analytics-156605.rush_app_bi.transactions_ledger_extended_full`
    where transaction_sub_category = 'TotalFee' and transaction_category = 'GamePlay' and platform = 'cashApp' and transaction_status = 'SUCCESS' and dt between "2022-09-11" and "2022-09-15" and game_ref_id like 'LUDO%' and amount=3
    order by updated_timestamp)
    """

    all_games = pd.read_gbq(query, use_bqstorage_api=True)

    all_games = all_games.to_dict('records')

    with open('../data/all_games_11_to_15.pkl', 'wb') as f:
        pickle.dump(all_games, f)

else:

    with open('../data/all_games_11_to_15.pkl', 'rb') as f:
        all_games = pickle.load(f)

In [3]:
with open('../data/delta_q_11_to_15.pkl', 'rb') as f:
    data = pickle.load(f)

In [4]:
live_scores = {}
live_scores1 = {}

for d in tqdm.tqdm(all_games):
    if d['game_ref_id'] in data and 'delta_q_1' in data[d['game_ref_id']]:
        dat = data[d['game_ref_id']]
        win = dat['score'][0] > dat['score'][1]
        pred = dat['delta_q_0'] > dat['delta_q_1']

        if dat['user_id_0'] not in live_scores:
            live_scores[dat['user_id_0']] = start_val
        if dat['user_id_1'] not in live_scores:
            live_scores[dat['user_id_1']] = start_val
        if dat['user_id_0'] not in live_scores1:
            live_scores1[dat['user_id_0']] = start_val
        if dat['user_id_1'] not in live_scores1:
            live_scores1[dat['user_id_1']] = start_val

        data[d['game_ref_id']]['q_score_0'] = live_scores[dat['user_id_0']]
        data[d['game_ref_id']]['q_score_1'] = live_scores[dat['user_id_1']]
        data[d['game_ref_id']]['score_score_0'] = live_scores1[dat['user_id_0']]
        data[d['game_ref_id']]['score_score_1'] = live_scores1[dat['user_id_1']]

        data[d['game_ref_id']]['dt'] = str(d['date'])

        if win == pred:
            live_scores[dat['user_id_0']] += dat['delta_q_0']
            live_scores[dat['user_id_1']] += dat['delta_q_1']

        live_scores1[dat['user_id_0']] += (dat['score'][0] - dat['score'][1])
        live_scores1[dat['user_id_1']] += (dat['score'][1] - dat['score'][0])

100%|██████████| 1441200/1441200 [00:13<00:00, 110264.50it/s]


In [5]:
win = []
q_pred = []
skill_pred = []
q_score_0 = []
q_score_1 = []

feats = []
feats_test = []
y = []
y_test = []

for dat in tqdm.tqdm(data):
    if 'delta_q_1' in data[dat]:
        if data[dat]['dt'] == '2022-09-15':
            win.append(data[dat]['score'][0] > data[dat]['score'][1])
            q_pred.append(data[dat]['q_score_0'] > data[dat]['q_score_1'])
            skill_pred.append((data[dat]['mu_0'] - 3*data[dat]['sigma_0']) > (data[dat]['mu_1'] - 3*data[dat]['sigma_1']))

            q_score_0.append(data[dat]['score_score_0'])
            q_score_1.append(data[dat]['score_score_1'])

        if data[dat]['dt'] != '2022-09-15':
            # feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
            #                 data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1']])
            y.append(data[dat]['score'][0] > data[dat]['score'][1])
            # feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['score_score_0'], 
            #                 data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['score_score_1']])
            feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1']])
        else:
            # feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
            #                 data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1']])
            y_test.append(data[dat]['score'][0] > data[dat]['score'][1])
            # feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['score_score_0'], 
            #                 data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['score_score_1']])
            feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1']])

100%|██████████| 916750/916750 [00:06<00:00, 141819.98it/s]


In [6]:
win = np.array(win).astype('int32')
q_pred = np.array(q_pred).astype('int32')
skill_pred = np.array(skill_pred).astype('int32')

feats = np.array(feats)
feats_test = np.array(feats_test)
y = np.array(y)
y_test = np.array(y_test)

In [7]:
np.sum(win == skill_pred)/len(win)

0.4848656356987887

In [8]:
np.sum(win == q_pred)*100/len(win)

49.7644532485476

In [9]:
q_score_0 = np.array(q_score_0)
q_score_1 = np.array(q_score_1)

bins = np.abs(q_score_0 - q_score_1)

In [10]:
u1 = []
tot = []
for i in range(25):
    idx = np.logical_and(bins>i*200, bins<(i+1)*200)
    temp = win[idx]
    u1.append(np.sum(temp[q_score_0[idx]>q_score_1[idx]]) + np.sum(temp[q_score_0[idx]<q_score_1[idx]]==0))
    tot.append(len(q_score_0[idx]))

In [11]:
import pandas as pd
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,27066,53818
1,18912,37497
2,12114,24202
3,7962,15620
4,5222,10200
5,3547,6976
6,2443,4720
7,1669,3274
8,1207,2324
9,857,1638


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [13]:
clf = Pipeline([('scaler', StandardScaler()), ('gbc', GradientBoostingClassifier(random_state=42, verbose=1))])
# clf = Pipeline([('scaler', StandardScaler()), ('gbc', LogisticRegression(random_state=42, verbose=1))])
clf = clf.fit(feats, y)

      Iter       Train Loss   Remaining Time 
         1           1.3797            3.73m
         2           1.3792            3.73m
         3           1.3788            3.65m
         4           1.3784            3.61m
         5           1.3780            3.56m
         6           1.3777            3.52m
         7           1.3774            3.48m
         8           1.3772            3.44m
         9           1.3768            3.39m
        10           1.3766            3.35m
        20           1.3747            2.97m
        30           1.3735            2.58m
        40           1.3727            2.19m
        50           1.3722            1.83m
        60           1.3717            1.46m
        70           1.3714            1.09m
        80           1.3711           43.58s
        90           1.3709           21.78s
       100           1.3707            0.00s


In [14]:
y_pred = clf.predict(feats_test)
y_probs = clf.predict_proba(feats_test)

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

def perf(ytrue, ypred):
    accu = accuracy_score(ytrue, ypred)
    prec = precision_score(ytrue, ypred)
    rec = recall_score(ytrue, ypred)
    return accu*100, prec*100, rec*100

In [16]:
acc, precision, recall = perf(y_test, y_pred)
print(acc, precision, recall)

54.52199823548016 54.91109795458452 88.56852416597357


In [17]:
bins = np.abs(y_probs[:,1]-0.5)

In [18]:
u1 = []
tot = []
for i in range(25):
    idx = np.logical_and(bins>i*0.05, bins<(i+1)*0.05)
    temp = y_test[idx]
    u1.append(np.sum(temp[y_probs[:,1][idx]>=0.5]==1) + np.sum(temp[y_probs[:,1][idx]<0.5]==0))
    tot.append(len(y_test[idx]))

In [19]:
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,60264,114394
1,30288,53439
2,6115,9987
3,1502,2274
4,87,122
5,3,3
6,0,0
7,0,0
8,0,0
9,0,0


In [20]:
clf = Pipeline([('scaler', StandardScaler()), ('gbc', LogisticRegression(random_state=42, verbose=1))])
clf = clf.fit(feats, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.92586D+05    |proj g|=  2.76765D+04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7      7      9      1     0     0   1.145D+00   4.891D+05
  F =   489135.41040046175     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished


In [21]:
y_pred = clf.predict(feats_test)
y_probs = clf.predict_proba(feats_test)

In [22]:
clf['gbc'].coef_

array([[-0.11093235,  0.06418526,  0.0041439 ,  0.11074142, -0.05209461,
        -0.01790263]])

In [23]:
# a = pd.DataFrame(np.concatenate((feats_test[:100], np.expand_dims(y_probs[:100,1],-1), np.expand_dims(y_test[:100],-1)),-1))

In [24]:
# a.to_csv('../data/temp.csv')

In [25]:
# pd.DataFrame(np.concatenate((np.expand_dims(clf['scaler'].mean_, -1),
#         (np.expand_dims(clf['scaler'].var_, -1)),
#         (np.expand_dims(np.squeeze(clf['gbc'].coef_), -1))), -1), columns = ['1', '2', '3'])

In [26]:
# np.sum(y_test[:100] == y_pred[:100])